In [55]:
from torch.utils.data import Dataset
import numpy as np
import os
from PIL import Image
from datasets.data_io import *
import scipy
import torch 
import cv2
import matplotlib.pyplot as plt

os.environ["CUDA_VISIBLE_DEVICES"]="0"

list_obj = ["bear", "buddha", "cow", "pot2", "reading"]


def read_img(filename):
        img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
        img = (img.astype(np.float32) - np.min(img)) / (np.max(img) - np.min(img))  #scale
        return img

In [29]:
def toImage(tensor, saving_directory, min_1 = 1450, max_1 = 1600):
    plt.imshow(tensor, vmin = min_1, vmax = max_1)
    plt.colorbar()
    plt.savefig(saving_directory)
    plt.close()

In [31]:

for obj_index in range(5):
    for view_index in range(20):
        sum_of_depthmap = 0
        for lighting_index in range(96):
            filename = list_obj[obj_index] + f"_view{view_index+1}" + f"_lighting{lighting_index}" + ".pt"
            depthmap = torch.load(f"/playpen-nas-ssd/xiaolong/project/MVSNet_pytorch/result/{filename}")
            sum_of_depthmap += depthmap["depth"]
        sum_of_depthmap /= 96
        saving_directory_img = "/playpen-nas-ssd/xiaolong/project/MVSNet_pytorch/result_combined_lighting/" + list_obj[obj_index] + f"_view{view_index+1}" + ".png"
        saving_directory_tensor = "/playpen-nas-ssd/xiaolong/project/MVSNet_pytorch/result_combined_lighting/" + list_obj[obj_index] + f"_view{view_index+1}" + ".pt"
        toImage(sum_of_depthmap, saving_directory_img, min_1 = 1450, max_1 = 1600)
        torch.save(sum_of_depthmap, saving_directory_tensor)

In [8]:
saving_directory_img

'/playpen-nas-ssd/xiaolong/project/MVSNet_pytorch/result_combined_lighting_notupscaled/bear_view1.png'

In [9]:
a = "reading_view20_lighting78.png"
b = a.split("_")
b[-1].split(".")

['lighting78', 'png']

In [7]:
def pooling(mat,ksize = (4,4),method='max',pad=False):
    '''Non-overlapping pooling on 2D or 3D data.

    <mat>: ndarray, input array to pool.
    <ksize>: tuple of 2, kernel size in (ky, kx).
    <method>: str, 'max for max-pooling, 
                   'mean' for mean-pooling.
    <pad>: bool, pad <mat> or not. If no pad, output has size
           n//f, n being <mat> size, f being kernel size.
           if pad, output has size ceil(n/f).

    Return <result>: pooled matrix.
    '''
    m, n = mat.shape[:2]
    ky,kx=ksize

    _ceil=lambda x,y: int(np.ceil(x/float(y)))

    if pad:
        ny=_ceil(m,ky)
        nx=_ceil(n,kx)
        size=(ny*ky, nx*kx)+mat.shape[2:]
        mat_pad=np.full(size,np.nan)
        mat_pad[:m,:n,...]=mat
    else:
        ny=m//ky
        nx=n//kx
        mat_pad=mat[:ny*ky, :nx*kx, ...]

    new_shape=(ny,ky,nx,kx)+mat.shape[2:]

    if method=='max':
        result=np.nanmax(mat_pad.reshape(new_shape),axis=(1,3))
    else:
        result=np.nanmean(mat_pad.reshape(new_shape),axis=(1,3))

    return result

In [56]:
obj_error = torch.ones((5,20))
for obj_index in range(5):
    for view_index in range(1, 21):
        if  view_index  < 10:
                gt_view = f"view_0{view_index}"
        else:
                gt_view = f"view_{view_index}"
        gt = np.load(f"/playpen-nas-ssd/xiaolong/data/DiLiGenT-MV/mvpmsData/{list_obj[obj_index]}PNG/depth/{gt_view}.npy")
        prediction = torch.load(f"/playpen-nas-ssd/xiaolong/project/MVSNet_pytorch/result_combined_lighting/{list_obj[obj_index]}_view{view_index}.pt")
        mask = torch.tensor(read_img(f"/playpen-nas-ssd/xiaolong/data/DiLiGenT-MV/mvpmsData/{list_obj[obj_index]}PNG/mask_depth/{gt_view}.png")[:, :-4])
        gt = torch.tensor(gt[:, :-4])
        gt = (gt < 0) + gt
        gt = torch.tensor(gt)
        gt = gt * mask
        difference = torch.abs(gt - prediction)
        save_directory_img = "/playpen-nas-ssd/xiaolong/project/MVSNet_pytorch/result_errormap/" + list_obj[obj_index] + f"_view{view_index}" + ".png"
        save_directory_tensor = "/playpen-nas-ssd/xiaolong/project/MVSNet_pytorch/result_errormap/" + list_obj[obj_index] + f"_view{view_index}" + ".pt"
        toImage(difference, save_directory_img, min_1 = 0, max_1 = 100)
        torch.save(difference, save_directory_tensor)
        obj_error[obj_index, view_index-1] = torch.sum(difference) / torch.count_nonzero(mask)


/tmp/ipykernel_260272/3603461707.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  gt = torch.tensor(gt)


In [57]:

a = obj_error.cpu().numpy()
np.savetxt("yeah.csv", a)

In [58]:
obj_error

tensor([[ 4.3947,  3.9124,  5.4983,  6.6486,  6.4028,  5.9202,  3.9855,  4.4135,
          4.4926,  4.9370,  4.6464,  3.9140,  3.1896,  4.6744,  6.8803, 10.7236,
         11.2102,  7.9253,  6.8133,  6.7411],
        [ 3.5168,  3.6152,  4.1245,  4.8840,  5.2177,  6.9052,  8.5578,  5.1742,
          3.3325,  3.3357,  3.4309,  3.8497,  5.6784,  7.1061,  7.1715,  5.8814,
          5.6710,  5.5074,  6.6406,  6.5281],
        [ 9.7015, 12.6548, 16.5676, 16.4373, 15.5530, 29.5888,  8.4015,  6.5955,
          6.6257,  7.3598,  7.8918,  9.1364,  9.8102,  8.7715, 11.5891, 23.9889,
         23.2645, 13.9336, 12.5872, 16.7200],
        [ 3.0618,  3.8760,  5.6505,  8.9872, 11.0102,  9.5670,  8.4240,  6.4321,
          6.0971,  5.6828,  5.4337,  5.4033,  7.2765,  7.1474,  6.8274,  9.6885,
         10.2918,  6.8654,  5.1656,  5.5246],
        [12.7976, 15.6977, 15.0285, 11.5030,  5.7863,  4.4326,  2.9557,  2.0441,
          3.2949,  3.4943,  3.2918,  5.1587,  7.7921,  8.0286,  6.1764,  5.4719,
      